<a href="https://colab.research.google.com/github/lindaperez/bk-covid19/blob/lperez_scraping/ETL_data_to_feed_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping:
*Web scraping is a term for various methods used to collect information from across the Internet. Generally, this is done with software that simulates human Web surfing to collect specified bits of information from different websites.*

Purpose: This file will update the database covid_db adding new patients and overwriting patients that already exist. This file will run from a cron demond every day at 11:00 PM PDT. 

### Installing Spark, Postgresql, Defining Environment Variables

In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (95.3 kB/s)
Reading package li

### - Postgresql installation

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-03 18:17:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.97MB/s    in 0.2s    

2022-07-03 18:17:38 (4.97 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



### - Creating Spark Session

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace
spark = SparkSession.builder.appName("download_data_to_feed_db").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### - Installing library to upzip the file

In [8]:
!pip install zipfile_deflate64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Scraping Process

### 1.- Download file from the website 

In [9]:
from pyspark import SparkFiles
url = "https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
spark.sparkContext.addFile(url)

## 2.- Unzip File 

In [11]:
file = SparkFiles.get("datos_abiertos_covid19.zip")
folder = 'datos_abiertos_new'
!unzip -o "$file" -d "$folder"

Archive:  /tmp/spark-8cb6e39e-5e19-4799-870e-6bfc9c06087d/userFiles-23ab1e78-1791-47a9-9ed3-688418d49d39/datos_abiertos_covid19.zip
  inflating: datos_abiertos_new/220702COVID19MEXICO.csv  


### 3.- Read the File unziped

In [12]:
# define the offset of the day to extract the data
from datetime import date,timedelta
today = date.today()- timedelta(days=1)
day = today.strftime('%y%m%d')
#Reading csv file using Pandas
#import pandas as pd 
#df = pd.read_csv('datos_abiertos_new/'+day+'COVID19MEXICO.csv',low_memory=False,engine='c')
filename = day+'COVID19MEXICO.csv'
df = spark.read.csv(filename, header=True, inferSchema=True)

In [14]:
df.head(2)

[Row(FECHA_ACTUALIZACION='2022-07-02', ID_REGISTRO='z3bf80', ORIGEN=2, SECTOR=12, ENTIDAD_UM=8, SEXO=2, ENTIDAD_NAC=8, ENTIDAD_RES=8, MUNICIPIO_RES=37, TIPO_PACIENTE=1, FECHA_INGRESO='2020-07-28', FECHA_SINTOMAS='2020-07-20', FECHA_DEF='9999-99-99', INTUBADO=97, NEUMONIA=2, EDAD=35, NACIONALIDAD=1, EMBARAZO=97, HABLA_LENGUA_INDIG=2, INDIGENA=2, DIABETES=2, EPOC=2, ASMA=2, INMUSUPR=2, HIPERTENSION=2, OTRA_COM=2, CARDIOVASCULAR=2, OBESIDAD=2, RENAL_CRONICA=2, TABAQUISMO=2, OTRO_CASO=2, TOMA_MUESTRA_LAB=1, RESULTADO_LAB=1, TOMA_MUESTRA_ANTIGENO=2, RESULTADO_ANTIGENO=97, CLASIFICACION_FINAL=3, MIGRANTE=99, PAIS_NACIONALIDAD='México', PAIS_ORIGEN='97', UCI=97),
 Row(FECHA_ACTUALIZACION='2022-07-02', ID_REGISTRO='zze974', ORIGEN=1, SECTOR=6, ENTIDAD_UM=24, SEXO=1, ENTIDAD_NAC=24, ENTIDAD_RES=24, MUNICIPIO_RES=35, TIPO_PACIENTE=1, FECHA_INGRESO='2021-02-28', FECHA_SINTOMAS='2021-02-20', FECHA_DEF='9999-99-99', INTUBADO=97, NEUMONIA=99, EDAD=34, NACIONALIDAD=1, EMBARAZO=2, HABLA_LENGUA_INDIG=2

In [16]:
# Temporal Folder where the file is located 
SparkFiles.get(filename)

'/tmp/spark-8cb6e39e-5e19-4799-870e-6bfc9c06087d/userFiles-23ab1e78-1791-47a9-9ed3-688418d49d39/220702COVID19MEXICO.csv'

In [17]:
# Renaming columns from spanish to english
df = df.selectExpr('FECHA_ACTUALIZACION AS data_file_updated', 'ID_REGISTRO AS id_patient',
                     'ORIGEN AS resp_monitoring','SECTOR AS type_institution','ENTIDAD_UM AS state_medical_unit',
                      'SEXO AS gender','ENTIDAD_NAC AS state_patient_birth','ENTIDAD_RES AS state_residence',
                      'MUNICIPIO_RES AS city_patient_birth','TIPO_PACIENTE AS type_patient',
                     'FECHA_INGRESO AS date_admitted','FECHA_SINTOMAS AS date_patient_symp',
                     'FECHA_DEF AS date_patient_death','INTUBADO AS intubated','NEUMONIA AS pneumonia',
                     'EDAD AS age','NACIONALIDAD AS resident',
                     'EMBARAZO AS pregnant','HABLA_LENGUA_INDIG AS indigenous_lang',
                     'INDIGENA AS indigenous','DIABETES AS diabetes',
                     'EPOC AS copd','ASMA AS asthma','INMUSUPR AS immunosup',
                     'HIPERTENSION AS hypertension','OTRA_COM AS another_complication',
                     'CARDIOVASCULAR AS cardiovascular','OBESIDAD AS obesity',
                     'RENAL_CRONICA AS renal_chronic','TABAQUISMO AS tobacco',
                     'OTRO_CASO AS closed_contanct','TOMA_MUESTRA_LAB AS lab_sample',
                     'RESULTADO_LAB AS lab_result','TOMA_MUESTRA_ANTIGENO AS antigen_sample',
                     'RESULTADO_ANTIGENO AS antigen_result','CLASIFICACION_FINAL AS final_class',
                     'MIGRANTE AS migrant','PAIS_NACIONALIDAD AS country_nationality',
                      'PAIS_ORIGEN AS country_patient_birth','UCI AS icu')

In [18]:
# Configure connection with database AWS RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://prod-covid-patients.cqbgcjbaetrj.us-west-1.rds.amazonaws.com:5432/covid_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Overwriting df to table in RDS
df.write.jdbc(url=jdbc_url, table='patient', mode=mode, properties=config) #5:55:39

In [29]:
# Testing
_select_sql = "(SELECT * FROM PATIENT P LIMIT 3) temp"
df_select = spark.read.jdbc(url=jdbc_url,table=_select_sql,properties=config)
df_select.show(5)

+-----------------+----------+---------------+----------------+------------------+------+-------------------+---------------+------------------+------------+-------------+-----------------+------------------+---------+---------+---+--------+--------+---------------+----------+--------+----+------+---------+------------+--------------------+--------------+-------+-------------+-------+---------------+----------+----------+--------------+--------------+-----------+-------+-------------------+---------------------+---+
|data_file_updated|id_patient|resp_monitoring|type_institution|state_medical_unit|gender|state_patient_birth|state_residence|city_patient_birth|type_patient|date_admitted|date_patient_symp|date_patient_death|intubated|pneumonia|age|resident|pregnant|indigenous_lang|indigenous|diabetes|copd|asthma|immunosup|hypertension|another_complication|cardiovascular|obesity|renal_chronic|tobacco|closed_contanct|lab_sample|lab_result|antigen_sample|antigen_result|final_class|migrant|cou

In [28]:
print("*********************************************************************")
print("The scraping finished successfully with ",df.count()," rows loadded ")
print("*********************************************************************")

*********************************************************************
The scraping finished successfully with  16638854  rows loadded 
*********************************************************************
